# CNN with TensorFlow

In [ ]:
# When using docker tensorflow/tensorflow:latest-juypter
!pip install --trusted-host pypi.org --trusted-host pypi.python.org --trusted-host files.pythonhosted.org scikit-learn pandas

### Initialize the Kernel

In [1]:
# mute Hardware optimzation messages
import os
os.environ["TF_CPP_MIN_LOG_LEVEL"] = '1'

In [2]:
# import required libraries
import tensorflow as tf
from IPython.display import Markdown, display

def printmd(string):
    display(Markdown('# <span style="color:red">'+string+'</span>'))

## 1st part: classify MNIST using simple model
This MNIST dataset is a collection of 60000 handwritten digits and 10000 testing samples from the much larger NIST dataset. They have been size-normalized and centered in a fixed-size image for developer training.

The first model is a simple Multi-layer perceptron (a type of Neural Network) to perform classification tasks.

In [4]:
# Import the dataset from TensorFlow
mnist = tf.keras.datasets.mnist
(x_train, y_train), (x_test, y_test) = mnist.load_data()

11490434/11490434 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step 


In [5]:
# Normalize the data to improve optimization performance
x_train, x_test = x_train /255.0, x_test /255.0

In [8]:
# Review the first few labels
print("categorical Labels")
print(y_train[0:5])

categorical Labels
[5 0 4 1 9]


In [10]:
# Encode the labels into a one-hot vector
y_train = tf.one_hot(y_train, 10)
y_test = tf.one_hot(y_test, 10)

# Review one-hot encoded labels
print("one hot encoded labels")
print(y_train[0:5])

one hot encoded labels
tf.Tensor(
[[0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]], shape=(5, 10), dtype=float32)


Check the size of the datasets

In [11]:
print("number of training examples:", x_train.shape[0])
print("number of test examples:", x_test.shape[0])

number of training examples: 60000
number of test examples: 10000


Shuffle the dataset to improve randomness on batching

In [12]:
train_ds = tf.data.Dataset.from_tensor_slices((x_train, y_train)).batch(50)
test_ds = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(50)

2025-03-25 09:23:38.701062: W external/local_xla/xla/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 376320000 exceeds 10% of free system memory.


### Convert 2D images to 1D Vector

In [ ]:
# use the tensorflow flatten class for the reduction
from tensorflow.keras.layers import Flatten
flatten = Flatten(dtype = 'float32')

print("original data shape")
print(x_train.shape)

print("flattened shape")
print(flatten(x_train).shape)

### Convolution: 2D operation with native python

The 2D convolution operation is defined as:

<font size="4">$$ I'= \sum\limits_{u,v} I(x-u,y-v)g(u,v) $$ </font> 

1. _Example 1_ <br />
    Applying 2D convolution on an image represented by a 3x3 matrix according to the function $$g = (\begin{matrix}-1 & 1\end{matrix})$$

In [ ]:
from scipy import signal as sg

I = [[255,  7,  3],
     [212,240,  4],
     [218,216,230],]

g = [[-1,1]]

print("Without zero padding\n")
print('{0} \n'.format(sg.convolve(I,g, "valid")))
# The 'valid' argument states that the output consists only of those 
# elements that do not rely on the zero-padding

print("With zero padding \.")
print(sg.convolve(I,g))

2. _Example 2_ <br />
    With a more difficult case where $g = [\begin{bmatrix}-1 & 1\end{bmatrix},\begin{bmatrix}2 & 3\end{bmatrix}]$

In [ ]:
g = [[-1, 1],
     [ 2, 3]]

print('With zero padding \n')
print('{0} \n'.format(sg.convolve(I,g,"full")))
# The output is the full discrete linear convolution of hte inputs.
# It will use zero to complete the input matrix

print('With zero padding(same) \n')
print('{0} \n'.format(sg.convolve(I, g, "same")))
# The output is teh full discrete linear convolution of the inputs.
# It will use zero to complete the input matrix

print('Without zero padding \n')
print(sg.convolve(I,g, 'valid'))
# The 'valid' argument states that the output consists only of those
# elements that do not rely on the zero-padding.

## Coding with TensorFlow

Assuming that we have a 10x10 image as input (tensor):

In [ ]:
# or as 4D tensor = [1,10,10,1] = [batch size, width, height, number of channels]
input = tf.Variable(tf.random.normal([1,10,10,1]))

and we have a 3x3 filter (tensor):

In [ ]:
# or as 4D tensor = [3,3,1,1] = [width, height, channels, number of filters]
filter = tf.Variable(tf.random.normal([3,3,1,1]))

If we were to process with zero padding in 'SAME' mode:

In [ ]:
op = tf.nn.conv2d(input, filter, strides=[1,1,1,1], padding='SAME')    

If we were to process without zero padding in 'VALID' mode:

In [ ]:
op2 = tf.nn.conv2d(input, filter, strides=[1,1,1,1], padding='VALID')

In [ ]:
print("Input \n")
print('{0} \n'.format(input.numpy()))
print("Filter/Kernel \n")
print('{0} \n'.format(filter.numpy()))
print("Result/Feature Map with padding \n")
print(op.numpy())
print('\n')
print("Result/Feature Map with valid positions \n")
print(op2.numpy())

## Convolution applied on images

In [ ]:
# import required libraries
from scipy import signal
import matplotlib.pyplot as plt
from PIL import Image

### Testing on bird image

In [ ]:
im = Image.open('bird.jpg')

image_gr = im.convert("L") 
# This method converts colour images into black and white using the
# ITU-R 601-2 Luma transform
print("\n Original type: %r \n\n" % image_gr)

# Convert image to a matrix with values from 0 to 255 (uint8)
arr = np.asarray(image_gr)
print("After conversion to numerical representation: \n\n %r" % arr)

### activate matplotlib for Ipython
%matplotlib inline

### plot image
imgplot = plt.imshow(arr)
imgplot.set_cmap('gray')
print("\n Input image converted to gray scale: \n")
plt.show(imgplot)

Let's start experimenting using an edge detector kernel

In [ ]:
kernel = np.array([[ 0, 1, 0],
                   [ 1,-4, 1],
                   [ 0, 1, 0],])
grad = signal.convolve2d(arr, kernel, mode='same', boundary='symm')

Generate a feature map

In [ ]:
%matplotlib inline
print('GRADIENT MAGNITUDE - Feature map')

fig, aux = plt.subplots(figsize=(10,10))
aux.imshow(np.absolute(grad), cmap='gray')

Enhance and normalize the image pixels onto a 0 to 1 scale

In [ ]:
type(grad)

grad_biases = np.absolute(grad) + 100
grad_biases[grad_biases > 255] = 255

Visualize the transformation

In [ ]:
%matplotlib inline
print('GRADIENT MAGNITUDE - Feature map')
fig, aux = plt.subplots(figsize=(10,10))
aux.imshow(np.absolute(grad_biases), cmap='gray')

### Testing on image of a digit

In [ ]:
im = Image.open("num3.jpg")

image_gr = im.convert("L")
print("\n Original type: %r \n\n" % image_gr)

# convert image to a matrix with values from 0 to 255 (uint8)
arr = np.asarray(image_gr)
print("After conversion to numerical representation: \n\n %r" % arr)

### activating matplotlib for Ipython
%matplotlib inline

### Plot image
fig, aux = plt.subplots(figsize=(10,10))
imgplot = plt.imshow(arr)
imgplot.set_cmap('gray')
print("\n Input image converted to gray scale: \n")
plt.show(imgplot)

Experiment using an edge detector kernel

In [ ]:
kernel = np.array([[ 0, 1, 0],
                   [ 1,-4, 1],
                   [ 0, 1, 0],])
grad = signal.convolve2d(arr, kernel, mode='same', boundary='symm')

# Plot transformed image
%matplotlib inline

print('GRADIENT MAGNITUDE - Feature map')

fig, aux = plt.subplots(figsize=(10,10))
aux.imshow(np.absolute(grad), cmap='gray')